##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
<td>
<a target="_blank" href="https://www.tensorflow.org/alpha/tutorials/text/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
</td><td>
<a target="_blank"  href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/text/text_generation.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/tensorflow/docs/blob/master/site/en/r2/tutorials/text/text_generation.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

This tutorial demonstrates how to generate text using a character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware acclerator > GPU*. If running locally make sure TensorFlow version >= 1.11.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/programmers_guide/keras) and [eager execution](https://www.tensorflow.org/programmers_guide/eager). The following is sample output when the model in this tutorial trained for 30 epochs, and started with the string "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf

import numpy as np
import os
import time

  Could not find a version that satisfies the requirement tensorflow-gpu==2.0.0-alpha0 (from versions: 0.12.1, 1.0.0, 1.0.1, 1.1.0rc0, 1.1.0rc1, 1.1.0rc2, 1.1.0)
No matching distribution found for tensorflow-gpu==2.0.0-alpha0
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


### Read the data

First, look in the text:

In [10]:
#actual_file = path_to_file
actual_file = './res/thiru_full'

# Read, then decode for py2 compat.
text = open(actual_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 90338 characters


In [11]:
text

'\n\nதிருக்குறள்\n\n1. அறத்துப்பால்\n1.1 கடவுள் வாழ்த்து\n\n1.1.1 கடவுள் வாழ்த்து \n\n\nஅகர முதல எழுத்தெல்லாம் ஆதி\nபகவன் முதற்றே உலகு.\t1\nகற்றதனால் ஆய பயனென்கொல் வாலறிவன்\nநற்றாள் தொழாஅர் எனின்.\t2\nமலர்மிசை ஏகினான் மாணடி சேர்ந்தார்\nநிலமிசை நீடுவாழ் வார்.\t3\nவேண்டுதல் வேண்டாமை இலானடி சேர்ந்தார்க்கு\nயாண்டும் இடும்பை இல.\t4\nஇருள்சேர் இருவினையும் சேரா இறைவன்\nபொருள்சேர் புகழ்புரிந்தார் மாட்டு.\t5\nபொறிவாயில் ஐந்தவித்தான் பொய்தீர் ஒழுக்க\nநெறிநின்றார் நீடுவாழ் வார்.\t6\nதனக்குவமை இல்லாதான் தாள்சேர்ந்தார்க் கல்லால்\nமனக்கவலை மாற்றல் அரிது.\t7\nஅறவாழி அந்தணன் தாள்சேர்ந்தார்க் கல்லால்\nபிறவாழி நீந்தல் அரிது.\t8\nகோளில் பொறியின் குணமிலவே எண்குணத்தான்\nதாளை வணங்காத் தலை.\t9\nபிறவிப் பெருங்கடல் நீந்துவர் நீந்தார்\nஇறைவன் அடிசேரா தார்.\t10\n1.1.2 வான்சிறப்பு\n\nவான்நின்று உலகம் வழங்கி வருதலால்\nதான்அமிழ்தம் என்றுணரற் பாற்று.\t11\nதுப்பார்க்குத் துப்பாய துப்பாக்கித் துப்பார்க்குத்\nதுப்பாய தூஉம் மழை.\t12\nவிண்இன்று பொய்ப்பின் விரிநீர் வியனுலகத்து\nஉள்நின்று உடற்றும் பசி.\t13\nஏரின் உழாஅர் உழ

In [12]:
# Take a look at the first 250 characters in text
print(text[:250])



திருக்குறள்

1. அறத்துப்பால்
1.1 கடவுள் வாழ்த்து

1.1.1 கடவுள் வாழ்த்து 


அகர முதல எழுத்தெல்லாம் ஆதி
பகவன் முதற்றே உலகு.	1
கற்றதனால் ஆய பயனென்கொல் வாலறிவன்
நற்றாள் தொழாஅர் எனின்.	2
மலர்மிசை ஏகினான் மாணடி சேர்ந்தார்
நிலமிசை நீடுவாழ் வார்.	3
வேண்டுத


In [13]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

62 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [14]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [15]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  ' ' :   2,
  ',' :   3,
  '.' :   4,
  '0' :   5,
  '1' :   6,
  '2' :   7,
  '3' :   8,
  '4' :   9,
  '5' :  10,
  '6' :  11,
  '7' :  12,
  '8' :  13,
  '9' :  14,
  '>' :  15,
  '?' :  16,
  'ஃ' :  17,
  'அ' :  18,
  'ஆ' :  19,
  ...
}


In [16]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\n\nதிருக்குறள்' ---- characters mapped to int ---- > [ 1  1 37 52 45 54 29 61 29 54 46 48 61]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [17]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])





த
ி
ர


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [18]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\n\nதிருக்குறள்\n\n1. அறத்துப்பால்\n1.1 கடவுள் வாழ்த்து\n\n1.1.1 கடவுள் வாழ்த்து \n\n\nஅகர முதல எழுத்தெல்லாம் ஆ'
'தி\nபகவன் முதற்றே உலகு.\t1\nகற்றதனால் ஆய பயனென்கொல் வாலறிவன்\nநற்றாள் தொழாஅர் எனின்.\t2\nமலர்மிசை ஏகினான் ம'
'ாணடி சேர்ந்தார்\nநிலமிசை நீடுவாழ் வார்.\t3\nவேண்டுதல் வேண்டாமை இலானடி சேர்ந்தார்க்கு\nயாண்டும் இடும்பை இல'
'.\t4\nஇருள்சேர் இருவினையும் சேரா இறைவன்\nபொருள்சேர் புகழ்புரிந்தார் மாட்டு.\t5\nபொறிவாயில் ஐந்தவித்தான் பொ'
'ய்தீர் ஒழுக்க\nநெறிநின்றார் நீடுவாழ் வார்.\t6\nதனக்குவமை இல்லாதான் தாள்சேர்ந்தார்க் கல்லால்\nமனக்கவலை மாற'


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [19]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [20]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\n\nதிருக்குறள்\n\n1. அறத்துப்பால்\n1.1 கடவுள் வாழ்த்து\n\n1.1.1 கடவுள் வாழ்த்து \n\n\nஅகர முதல எழுத்தெல்லாம் '
Target data: '\nதிருக்குறள்\n\n1. அறத்துப்பால்\n1.1 கடவுள் வாழ்த்து\n\n1.1.1 கடவுள் வாழ்த்து \n\n\nஅகர முதல எழுத்தெல்லாம் ஆ'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [21]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 1 ('\n')
  expected output: 1 ('\n')
Step    1
  input: 1 ('\n')
  expected output: 37 ('த')
Step    2
  input: 37 ('த')
  expected output: 52 ('ி')
Step    3
  input: 52 ('ி')
  expected output: 45 ('ர')
Step    4
  input: 45 ('ர')
  expected output: 54 ('ு')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [22]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [23]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [24]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [25]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next character:

![A drawing of the data passing through the model](https://tensorflow.org/tutorials/alpha/text/images/text_generation_training.png)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 62) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           15872     
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 62)            63550     
Total params: 5,326,398
Trainable params: 5,326,398
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [29]:
sampled_indices

array([ 0, 25, 52, 17, 50, 34, 31, 32, 33, 16, 59, 51, 51, 47, 50, 15, 46,
       11, 25,  7, 31, 35, 47, 31, 33,  1, 57, 23, 24, 13, 12, 61, 10, 55,
       16,  8, 35,  2, 33, 54,  1, 34, 25, 51, 17, 52,  9, 20, 60,  1, 52,
       32,  4,  0, 50, 28, 26, 30, 58, 17, 30, 31, 44, 43, 51, 37, 47,  9,
       14, 18, 42, 17, 26, 44, 11,  7, 34, 11, 58, 13, 15, 22, 18, 12, 12,
       15, 53, 43, 16, 14,  8, 27, 35, 31, 22, 25, 34, 42, 39, 20])

Decode these to see the text predicted by this untrained model:

In [30]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'இடம்.\t1123\nவாழ்தல் உயிர்க்கன்னள் ஆயிழை சாதல்\nஅதற்கன்னள் நீங்கும் இடத்து.\t1124\nஉள்ளுவன் மன்யான் மறப்ப'

Next Char Predictions: 
 '\tஏிஃவ\u0ba1சஞட?ொாாலவ>ற6ஏ2ச\u0ba2லசட\nேஊஎ87்5ூ?3\u0ba2 டு\n\u0ba1ஏாஃி4இோ\nிஞ.\tவஓஐஙைஃஙசயமாதல49அபஃஐய62\u0ba16ை8>உஅ77>ீம?93ஒ\u0ba2சஉஏ\u0ba1ப\u0ba7இ'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_softmax_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [31]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 62)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.12678


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [32]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [33]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [34]:
EPOCHS=10

In [35]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
13/13 [==============================] - 151s 12s/step - loss: 3.6564
Epoch 2/10
13/13 [==============================] - 145s 11s/step - loss: 3.3625
Epoch 3/10
13/13 [==============================] - 99s 8s/step - loss: 3.2218
Epoch 4/10
13/13 [==============================] - 93s 7s/step - loss: 2.7948
Epoch 5/10
13/13 [==============================] - 90s 7s/step - loss: 2.4344
Epoch 6/10
13/13 [==============================] - 92s 7s/step - loss: 2.3283
Epoch 7/10
13/13 [==============================] - 90s 7s/step - loss: 2.2249
Epoch 8/10
13/13 [==============================] - 94s 7s/step - loss: 2.1553
Epoch 9/10
13/13 [==============================] - 98s 8s/step - loss: 2.1041
Epoch 10/10
13/13 [==============================] - 97s 7s/step - loss: 2.0454


## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [36]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [37]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            15872     
_________________________________________________________________
unified_lstm_1 (UnifiedLSTM) (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 62)             63550     
Total params: 5,326,398
Trainable params: 5,326,398
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](https://tensorflow.org/tutorials/alpha/text/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [39]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [40]:
print(generate_text(model, start_string=u"எரியால்"))

எரியால்மு உய்வார்கு
வல்று.	123
4.4 பிகுஞ்தை இக்றெத வேர்கது அடல்.	123
ஒலழ்சா ஊகேப் பேளிர்கி வெய்றார் எறவ்நோண்ப் என்பதன் இலரிக்
வல்னு.	8141
2.2. வவிழுடவர்கும் ஊணர்டும் அடியவுய்க் யன்றும் 
பகிக்கு.	306
வண்சகவப் பொல்செர்க்கு இன்சில் சொல்.	1419
மாணம் வருவந்த கண்லாம்வஞ்
நைனிறா
தவர்யு ஊ.	141
உவபினா வாந்கல் ஒறுந்கவடி நீகந்கல்
இன்றும் பெவிர்.	706
நெயழப்குமை
வான்ய மறைம்பறு
யார்டாதும் கண்குமை
உம்.	97 நாணியன்தார் இர்ன் துபததக் இனன்.	759
இளழைவூம்ன் என்னடு இழைர்குசு வாருக்சொல்
சொல்ளி அறநி3.	436
வனைநற்க்ப் படைஞெனும் வர்பதியால்

யான் கடையறர் மின்
இடைக்கின் பெறவு வினிம்.	1407
வகுந்படன் தகண்செரி நான்துவை ஒழிலாப்
பென்ளத பிறொர்கின் அன்டாகாரும் 
யாரியி உனமை
பட்த விள்.	151
மறைமைன் பொல்பை.	1001
பதர்கொடுத் தல்படுன்
ஈத்னுப்பி நறினன் நீர்.	827
1.3.6 நலளிச்சி துயர்தகம் நாராண் ஒனினு.	609
ஓமகம் தனறெங்குன் பெல்லாரைடொச் கூகு தவருவடிற் தண்டு.	1612
இணண்சிச் செயயியாக் கூ்துத்டோய் தும்.	506
உடைடறியுங் குலாவன் ஆன்டு படனின்பார்
கோடல்
கிணும்பேன் பொகுகூம் உமற்இருக்குக் கறு.	534
ஊக்கதவஙுவைந்க மால அினைந்தும் தூழைத்தவழுள்
நல்.

The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement _curriculum learning_ to help stabilize the model's open-loop output.

We will use `tf.GradientTape` to track the gradiends. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The procedure works as follows:

* First, initialize the RNN state. We do this by calling the `tf.keras.Model.reset_states` method.

* Next, iterate over the dataset (batch by batch) and calculate the *predictions* associated with each.

* Open a `tf.GradientTape`, and calculate the predictions and loss in that context.

* Calculate the gradients of the loss with respect to the model variables using the `tf.GradientTape.grads` method.

* Finally, take a step downwards by using the optimizer's `tf.train.Optimizer.apply_gradients` method.



In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

W0304 03:54:08.030432 140067035297664 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f63635efe80>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [0]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.174627780914307
Epoch 1 Batch 100 Loss 2.333711862564087
Epoch 1 Loss 2.0831
Time taken for 1 epoch 15.117910146713257 sec

Epoch 2 Batch 0 Loss 2.150496244430542
Epoch 2 Batch 100 Loss 1.8478351831436157
Epoch 2 Loss 1.7348
Time taken for 1 epoch 14.401937007904053 sec

Epoch 3 Batch 0 Loss 1.8013415336608887
Epoch 3 Batch 100 Loss 1.6072556972503662
Epoch 3 Loss 1.5668
Time taken for 1 epoch 14.415359258651733 sec

Epoch 4 Batch 0 Loss 1.6106206178665161
Epoch 4 Batch 100 Loss 1.478020191192627
Epoch 4 Loss 1.4673
Time taken for 1 epoch 14.300090312957764 sec

Epoch 5 Batch 0 Loss 1.5047727823257446
Epoch 5 Batch 100 Loss 1.3985247611999512
Epoch 5 Loss 1.3992
Time taken for 1 epoch 14.128910779953003 sec

Epoch 6 Batch 0 Loss 1.4343167543411255
Epoch 6 Batch 100 Loss 1.3426867723464966
Epoch 6 Loss 1.3441
Time taken for 1 epoch 13.973440170288086 sec

Epoch 7 Batch 0 Loss 1.3767048120498657
Epoch 7 Batch 100 Loss 1.297922968864441
Epoch 7 Loss 1.2969
Time take